In [3]:
import xarray as xr
import numpy as np
from skimage.measure import label
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

ds_prcp = xr.open_dataset(f"/scratch/nf33/hk25_LSP/UK/Darwin/pr_hourly.nc")
domain_area = 200.0 * 200.0 

# Define thresholds
pr_th_list = [0.5, 1, 2, 3, 4, 5]

for pr_th in pr_th_list:
    num_obj_arr = np.zeros(len(ds_prcp.time))
    tot_area_arr = np.zeros(len(ds_prcp.time))
    mean_obj_area_arr = np.zeros(len(ds_prcp.time))
    area_frac_arr = np.zeros(len(ds_prcp.time))
    cvt_mean_prcp_arr = np.zeros(len(ds_prcp.time))
    tot_mean_prcp_arr = np.zeros(len(ds_prcp.time))
    cvt_tot_prcp_arr = np.zeros(len(ds_prcp.time))

    for its in range(0, len(ds_prcp.time)):
        prcp = ds_prcp["pr"].isel(time=its) * 3600
        cvt_prcp = np.sum(prcp.values[prcp.values >= pr_th])
        cv_obj = prcp.copy().fillna(0)
        cv_obj.values[cv_obj.values < pr_th] = 0
        cv_obj.values[cv_obj.values >= pr_th] = 6
        label_arr = label(cv_obj)
        unique_label = np.unique(label_arr)
        num_obj_arr[its] = len(unique_label) - 1

        ind_obj_area = np.zeros(len(unique_label) - 1)
        for i in unique_label:
            if i == 0:
                continue
            else:
                ind_obj_area[i - 1] = np.sum(label_arr == i) * 16

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning)
            tot_area_arr[its] = np.sum(ind_obj_area)
            if num_obj_arr[its] > 0:
                mean_obj_area_arr[its] = tot_area_arr[its] / num_obj_arr[its]
            else:
                mean_obj_area_arr[its] = np.nan
            area_frac_arr[its] = tot_area_arr[its] / domain_area

            if tot_area_arr[its] > 0:
                cvt_mean_prcp_arr[its] = cvt_prcp / tot_area_arr[its]
            else:
                cvt_mean_prcp_arr[its] = np.nan
            tot_mean_prcp_arr[its] = cvt_prcp / domain_area
            cvt_tot_prcp_arr[its] = cvt_prcp

    # Save for this threshold
    out = xr.Dataset(
        {
            "num_obj": (("time"), num_obj_arr),
            "tot_area": (("time"), tot_area_arr),
            "mean_obj_area": (("time"), mean_obj_area_arr),
            "area_frac": (("time"), area_frac_arr),
            "cvt_mean_prcp": (("time"), cvt_mean_prcp_arr),
            "tot_mean_prcp": (("time"), tot_mean_prcp_arr),
            "cvt_tot_prcp": (("time"), cvt_tot_prcp_arr),
        },
        coords={
            "time": ds_prcp.time.values,
        },
    )
    output_path = f"/scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_{pr_th}_hourly.nc"
    out.to_netcdf(output_path)
    print(f"Saved output for pr_th = {output_path}")
    out.close()


ds_prcp.close()


Saved output for pr_th = /scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_0.5_hourly.nc
Saved output for pr_th = /scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_1_hourly.nc
Saved output for pr_th = /scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_2_hourly.nc
Saved output for pr_th = /scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_3_hourly.nc
Saved output for pr_th = /scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_4_hourly.nc
Saved output for pr_th = /scratch/nf33/hk25_LSP/UK/Darwin/number_size_prth_5_hourly.nc
